<a href="https://colab.research.google.com/github/NimrodDev/TensorFlow/blob/main/RNN_Play_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 2.x
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [2]:
# This is our dataset
path_to_file = tf.keras.utils.get_file('shakespeare.txt','https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [3]:
# Here we look at the content of the file
# Read, then decode for py2 compat
text = open(path_to_file, 'rb').read().decode(encoding= 'utf-8')
# Length of text is the number of characters in it
print('Length of the text: {} characters'.format(len(text)))

Length of the text: 1115394 characters


In [4]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [5]:
vocab = sorted(set(text))
# Here we create a map from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
  return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)

In [6]:
# Lets look at how part of our text is encoded
print("Text:", text[:13])
print("Encoded:", text_to_int(text[:13]))

Text: First Citizen
Encoded: [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [7]:
def int_to_text(ints):
  try:
    ints = ints.numpy()
  except:
    pass
  return ''.join(idx2char[ints])

print(int_to_text(text_as_int[:13]))

First Citizen


In [8]:
# Here we create training
seq_length = 100 # Length of sequence for a training example
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [9]:
sequences = char_dataset.batch(seq_length+1, drop_remainder= True)

In [10]:
def split_input_target(chunk): # For the example: Hello
  input_text =chunk[:-1] # hell
  target_text = chunk[1:] # ello
  return input_text, target_text # hell, ello


dataset = sequences.map(split_input_target) # We use map to apply the above function to every entry


In [11]:
for x, y in dataset.take(2):
  print("\n\nEXAMPLE\n")
  print("INPUT")
  print(int_to_text(x))
  print("\nOUTPUT")
  print(int_to_text(y))



EXAMPLE

INPUT
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You

OUTPUT
irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 


EXAMPLE

INPUT
are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you 

OUTPUT
re all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k


In [12]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab) # Vocab is number of unique characters
EMBEDDING_DIM = 256
RNN_UNITS = 1024

#Buffer size to shuffle the dataset
'''TF data is designed to work with possibly infinte sequence
So it doesn't attempt to shuffle the entire sequence in memory, instead.
It maintains a buffer in which it shuffles elements.'''

BUFFER_SIZE = 10000
data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder= True)

In [13]:
# Here we build the model
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
      tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                batch_input_shape=[batch_size, None]),
      tf.keras.layers.LSTM(rnn_units,
                           return_sequences= True,
                           stateful= True,
                           recurrent_initializer= 'glorot_uniform'),
      tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(VOCAB_SIZE,EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           16640     
                                                                 
 lstm (LSTM)                 (64, None, 1024)          5246976   
                                                                 
 dense (Dense)               (64, None, 65)            66625     
                                                                 
Total params: 5330241 (20.33 MB)
Trainable params: 5330241 (20.33 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [14]:
for input_example_batch, target_example_batch in data.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape) # Prints out the output shape

(64, 100, 65)


In [15]:
# We can see that prediction is an array of 64 arrays, one for each entry in the batch
print(len(example_batch_predictions))
print(example_batch_predictions)


64
tf.Tensor(
[[[-9.53555107e-04 -5.43494057e-03  1.66971143e-03 ...  3.53426463e-03
    1.32870628e-03  1.11273467e-03]
  [ 1.89250568e-03 -2.20255926e-03 -2.99942167e-03 ...  4.84376727e-03
    1.16049789e-03 -1.87275698e-03]
  [ 2.60339747e-03 -6.33168686e-03 -8.92124022e-04 ...  2.36065476e-03
    3.17339203e-03 -8.32510088e-03]
  ...
  [-2.86091212e-03 -7.13585760e-05 -2.26686499e-03 ...  1.33764395e-03
    3.73524521e-03  1.63650699e-03]
  [-1.28315436e-03  5.95728587e-03  1.00507052e-03 ...  4.30892082e-03
    5.91445388e-03  2.60841614e-03]
  [-7.33029656e-03  2.13686936e-03  3.06039560e-03 ... -5.12046041e-04
    3.15929530e-03  5.04571106e-03]]

 [[-4.67534550e-03  3.27738002e-04 -2.94020912e-03 ... -3.76581121e-03
    1.92881876e-03  1.71950355e-03]
  [ 1.05416728e-03  3.61941382e-03  1.62655488e-05 ... -7.37315277e-03
    2.23714672e-03  1.88839587e-03]
  [ 5.47171850e-03  6.63154759e-03  2.91790068e-03 ... -9.27686412e-03
    3.55854374e-03  1.97287882e-03]
  ...
  [-2.165

In [16]:
# Lets examine one prediction
pred = example_batch_predictions[0]
print(len(pred))
print(pred)
# Notice this is a 2d array of length 100, where each interior array is the prediction for the next character at each time ste

100
tf.Tensor(
[[-9.5355511e-04 -5.4349406e-03  1.6697114e-03 ...  3.5342646e-03
   1.3287063e-03  1.1127347e-03]
 [ 1.8925057e-03 -2.2025593e-03 -2.9994217e-03 ...  4.8437673e-03
   1.1604979e-03 -1.8727570e-03]
 [ 2.6033975e-03 -6.3316869e-03 -8.9212402e-04 ...  2.3606548e-03
   3.1733920e-03 -8.3251009e-03]
 ...
 [-2.8609121e-03 -7.1358576e-05 -2.2668650e-03 ...  1.3376439e-03
   3.7352452e-03  1.6365070e-03]
 [-1.2831544e-03  5.9572859e-03  1.0050705e-03 ...  4.3089208e-03
   5.9144539e-03  2.6084161e-03]
 [-7.3302966e-03  2.1368694e-03  3.0603956e-03 ... -5.1204604e-04
   3.1592953e-03  5.0457111e-03]], shape=(100, 65), dtype=float32)


In [17]:
# FInally well look at a prediction at the first timestep
time_pred = pred[0]
print(len(time_pred))
print(time_pred)
# Its 65 values representing the probability of each character accuring next

65
tf.Tensor(
[-9.5355511e-04 -5.4349406e-03  1.6697114e-03 -1.7866559e-03
  2.4600828e-03 -1.6624543e-03  9.9329266e-04 -1.3200623e-03
  6.3031825e-05  6.5862946e-03  6.7458041e-03  3.4296159e-03
  1.0350549e-03 -2.9278686e-03 -2.0644106e-03  8.3867274e-04
  2.5963085e-03 -7.9236284e-04 -5.0064488e-03  5.0838920e-04
 -1.5255702e-03 -2.0408265e-03  8.3077629e-04  7.4615318e-04
  1.5583599e-03  2.5007981e-03  5.2456750e-04  3.9360048e-03
  4.6492489e-03 -3.7979968e-03  5.6849309e-04 -3.8482677e-03
 -1.9216794e-03  5.0619314e-03  5.3372089e-04  8.0431076e-03
 -4.5262335e-04 -6.0866482e-04  7.5346325e-03  3.7554770e-03
  1.5133934e-03  5.0661284e-03  2.6127354e-03  1.8597584e-03
  1.5738355e-04 -9.2232286e-04  9.4832521e-04  4.4974163e-03
 -1.2398171e-03 -2.7429275e-04 -5.1549561e-03  9.4863790e-04
 -4.1169934e-03  4.7797733e-04  3.2755202e-03 -8.5362117e-04
  4.2046560e-03  4.1138497e-04  5.6123943e-04  2.0713487e-03
 -2.7899221e-03  4.6606776e-03  3.5342646e-03  1.3287063e-03
  1.112734

In [18]:
# If we want to determine the predicted character we need to sample the output distribution
sampled_indices = tf.random.categorical(pred, num_samples=1)

# Now we can reshape that array and convert all the integers to numbers to see the actual characters
sampled_indices = np.reshape(sampled_indices, (1, -1))[0]
predicted_chars = int_to_text(sampled_indices)

predicted_chars # This is what the model predicted for training sequence 1

'lbIyZ:vbvoO,hNgxhfc3XevP XMh$iaNdU-pHwMVqfdIKha-EQH\niEwEjWXXHDr!LZmpABRLcYqApmEaUOuzQIr WKiTz?cz$Hdu'

In [19]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [20]:
# Here we compile our model
model.compile(optimizer='adam', loss=loss)

In [21]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback= tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only= True)

In [22]:
# Finally we start training our model
history = model.fit(data, epochs=10, callbacks=[checkpoint_callback])

Epoch 1/10
172/172 [==============================] - 21s 84ms/step - loss: 2.5471
Epoch 2/10
172/172 [==============================] - 16s 82ms/step - loss: 1.8635
Epoch 3/10
172/172 [==============================] - 16s 82ms/step - loss: 1.6207
Epoch 4/10
172/172 [==============================] - 15s 80ms/step - loss: 1.4928
Epoch 5/10
172/172 [==============================] - 14s 75ms/step - loss: 1.4134
Epoch 6/10
172/172 [==============================] - 16s 77ms/step - loss: 1.3568
Epoch 7/10
172/172 [==============================] - 15s 81ms/step - loss: 1.3129
Epoch 8/10
172/172 [==============================] - 16s 82ms/step - loss: 1.2733
Epoch 9/10
172/172 [==============================] - 15s 78ms/step - loss: 1.2371
Epoch 10/10
172/172 [==============================] - 15s 78ms/step - loss: 1.2022


In [23]:
# Here we load our model
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size = 1)

In [24]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [28]:
# We can load any checkpoint we went want by specifying the exact file to load
checkpoint_num = 10

model.load_weights(tf.train.load_checkpoint("./training_checkpoints/ckpt_" + str(checkpoint_num)))
model.build(tf.TensorShape([1, None]))

AttributeError: 'tensorflow.python.util._pywrap_checkpoint_reader.C' object has no attribute 'endswith'

In [29]:
def generate_text(model, start_string):
  # Evaluation step
  # Number of characters to generate
  num_generate = 800

  #Converting our start string to numbers(vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures result in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch_size == 1
  model.reset_states()
  for i in range(num_generate):
    predictions = model(input_eval)
    # Remove the batch dimension
    predictions = tf.squeeze(predictions, 0)

    # Using a categorical distribution to predict the character returned by the model
    predictions =predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

    '''We pass the predicted character as the next input to the model'''
    input_eval = tf.expand_dims([predicted_id], 0)

    text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [30]:
inp = input("Type a starting string:")
print(generate_text(model, inp))

Type a starting string:romeo
romeous great sovereign:
High power will swear by rooting kings,
It shuds i' the sun hath thy custombas's the bed the imprison!

WARWICK:
We are se; commends thee in the beggard
Calling of CarubaMa mayarnight hide thee,
And enter gait wither and their tribbed soul,
And in the gross his power to?
Was I think so, sir, and as his wrong
As comes your ears and night
That noble and will I kind their ey alives
For heaven and such a slaughter
Deserve all royalty; good careless dog!
Come, sirs, let's take off in himself my rebellion;
A meeting thing that he is s death, this is my loving bre.

RIVERS:
What save your brood, my lord.

Second Saint well, now what is bold, that Julium
As yours I'll sullen in this new head which be denied them.

YORK:
What should in jession will hear pardon?

LADY CAPULET:
Fe
